In [40]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

In [41]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    return x

def encoder_block(inputs, num_filters):
    s = conv_block(inputs, num_filters)
    p = MaxPool2D((2,2))(s)
    return s,p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    """ Input layer"""
    inputs = Input(input_shape)
    
    """ Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    """ Bottleneck"""
    b1 = conv_block(p4, 1024)
    
    """ Decoder"""
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    """ Output layer """
    outputs = Conv2D(1,1, padding="same", activation="sigmoid")(d4)
    
    model = Model(inputs, outputs, name="UNET")
    return model


model = build_unet((256,256,3))
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 256, 256, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 256, 256, 64  256        ['conv2d_19[0][0]']              
 ormalization)                  )                                                              

 ormalization)                  )                                                                 
                                                                                                  
 activation_26 (Activation)     (None, 16, 16, 1024  0           ['batch_normalization_26[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_28 (Conv2D)             (None, 16, 16, 1024  9438208     ['activation_26[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_27 (BatchN  (None, 16, 16, 1024  4096       ['conv2d_28[0][0]']              
 ormalization)                  )                                                                 
          

                                                                                                  
 conv2d_35 (Conv2D)             (None, 256, 256, 64  73792       ['concatenate_7[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 256, 256, 64  256        ['conv2d_35[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_34 (Activation)     (None, 256, 256, 64  0           ['batch_normalization_34[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_36

In [3]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union+1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [42]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [66]:
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def shuffling(x,y):
    x,y = shuffle(x, y, random_state=42)
    
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path,cv2.IMREAD_GRAYSCALE) # (256,256) --> (256, 256, 1)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    
    x, y = tf.numpy_function(_parse, [x, y],[tf.float32, tf.float32])
    x.set_shape([H,W,3])
    y.set_shape([H,W,1])
    return x, y

def tf_dataset(X, Y, batch =8):
    dataset = tf.data.Dataset.from_tensor_slices((X,Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

    
def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "Original/","*/" "*.jpg")))
    masks = sorted(glob(os.path.join(path, "ground_truth/","*/", "*.jpg")))
    size = int(len(images) * split)
    
    train_x, valid_x, train_y, valid_y = train_test_split(images, masks, test_size=split, random_state=42)
    train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=split, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir("files")

""" Hyperparameters"""
batch_size = 8
lr = 1e-4
num_epochs = 10
model_path = "segmodel.h5"
csv_path = "data.csv"

"""  Dataset """
dataset_path = "C:\\Users\\Albayinah Rasheed\\Downloads\\GrTh\\GrTh\\"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)    
train_x, train_y = shuffle(train_x, train_y)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

#ds = (1,2,3,4,5)
#bs = 2
#n = len(ds)//bs = 2
#[1,2], [3,4], [1]

train_steps = (len(train_x)//batch_size)
valid_steps = (len(valid_x)//batch_size)

if len(train_x) % batch_size !=0:
    train_steps += 1
    
if len(valid_x) % batch_size !=0:
    valid_steps += 1
    
"""Model"""
model = build_unet((H, W, 3))
metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)

callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        #callbacks=callbacks
)

Train: 732 - 732
Valid: 229 - 229
Test: 184 - 184
Epoch 1/10
92/92 [==============================] - 1194s 13s/step - loss: 0.2734 - dice_coef: 0.5200 - iou: 0.3580 - recall_5: 0.7729 - precision_5: 0.8000 - val_loss: 0.6103 - val_dice_coef: 0.1967 - val_iou: 0.1093 - val_recall_5: 0.0103 - val_precision_5: 0.1362
Epoch 2/10
92/92 [==============================] - 1080s 12s/step - loss: 0.1454 - dice_coef: 0.6516 - iou: 0.4846 - recall_5: 0.6903 - precision_5: 0.9388 - val_loss: 0.4180 - val_dice_coef: 0.1128 - val_iou: 0.0598 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00
Epoch 3/10
92/92 [==============================] - 992s 11s/step - loss: 0.1209 - dice_coef: 0.6894 - iou: 0.5276 - recall_5: 0.6757 - precision_5: 0.9517 - val_loss: 0.5590 - val_dice_coef: 0.1398 - val_iou: 0.0754 - val_recall_5: 0.0861 - val_precision_5: 0.0758
Epoch 4/10
92/92 [==============================] - 997s 11s/step - loss: 0.1066 - dice_coef: 0.7193 - iou: 0.5631 - recall_5: 0.6613 - precis

In [67]:
model.save('SegmentationModel.h5')

In [83]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score


H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)   ## (1, 256, 256, 3)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.int32)
    return ori_x, x

def save_result(ori_x, ori_y, y_pred, save_path):
    line = np.ones((H, 10, 3)) * 255
    
    ori_y = np.expand_dims(ori_y, axis=-1) ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256,256,3)
    
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255.0
    
    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv2.imwrite(save_path, cat_images)
    
create_dir("results")

"""load model"""
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
    model = tf.keras.models.load_model("SegmentationModel.h5")
    model.summary()
    
"""Dataset"""
dataset_path = "C:\\Users\\Albayinah Rasheed\\Downloads\\GrTh\\GrTh\\"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

"""Prediction and metrics values"""
SCORE = []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    name = x.split("\\")[-1]
    
    """ Reading image and mask"""
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)
    
    """Prediction"""
    y_pred = model.predict(x)[0] > 0.5 
    y_pred = np.squeeze(y_pred, axis=-1)
    y_pred = y_pred.astype(np.int32)
    
    save_path = f"results/{name}"
    save_result(ori_x, ori_y, y_pred, save_path)
    
    """ Flattening the numpy arrays."""
    y = y.flatten()
    y_pred = y_pred.flatten()
    
    """ Calculating metrics values """
    acc_value = accuracy_score(y, y_pred)
    f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
    jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
    precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
    SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])
    
""" Metrics values """
score = [s[1:]for s in SCORE]
score = np.mean(score, axis=0)
print(f"Accuracy: {score[0]:0.5f}")
print(f"F1: {score[1]:0.5f}")
print(f"Jaccard: {score[2]:0.5f}")
print(f"Recall: {score[3]:0.5f}")
print(f"Precision: {score[4]:0.5f}")

""" Saving all the results """
df = pd.DataFrame(SCORE, columns= ["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score.csv")

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_133 (Conv2D)            (None, 256, 256, 64  1792        ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_126 (Batch  (None, 256, 256, 64  256        ['conv2d_133[0][0]']             
 Normalization)                 )                                                              

                                )                                                                 
                                                                                                  
 batch_normalization_134 (Batch  (None, 16, 16, 1024  4096       ['conv2d_141[0][0]']             
 Normalization)                 )                                                                 
                                                                                                  
 activation_134 (Activation)    (None, 16, 16, 1024  0           ['batch_normalization_134[0][0]']
                                )                                                                 
                                                                                                  
 conv2d_142 (Conv2D)            (None, 16, 16, 1024  9438208     ['activation_134[0][0]']         
                                )                                                                 
          

                                                                                                  
 concatenate_31 (Concatenate)   (None, 256, 256, 12  0           ['conv2d_transpose_31[0][0]',    
                                8)                                'activation_127[0][0]']         
                                                                                                  
 conv2d_149 (Conv2D)            (None, 256, 256, 64  73792       ['concatenate_31[0][0]']         
                                )                                                                 
                                                                                                  
 batch_normalization_142 (Batch  (None, 256, 256, 64  256        ['conv2d_149[0][0]']             
 Normalization)                 )                                                                 
                                                                                                  
 activatio

100%|████████████████████████████████████████████████████████████████████████████████| 184/184 [01:51<00:00,  1.65it/s]

Accuracy: 0.97567
F1: 0.87087
Jaccard: 0.78919
Recall: 0.91621
Precision: 0.84601
